# Natural Disaster Data

## Task:

The goal is to convert the data into the following formats for later use.
Along the way, this notebook does some data-preparation


### Disaster-All
disaster/disaster-all:
Columns: disaster_no, year, subgroup, type, total_deaths, dis_mag_value, dis_mag_scale, start_year, end_year
Other interesting columns?


### Disaster-Global
disaster/disaster-global.csv
Columns: year, subgroup, type, total_deaths


### Disaster-Region
disaster/disaster-region.csv
Columns: region_code, region_name, year, subgroup, type, total_deaths
Calculate from country data, use UN Dataset to assign region to each country


### Disaster-Country
disaster/disaster-country.csv
Columns: year, country_code, country_name,  subtype, type, total_deaths

Prefix: dis


## Setup & Imports

In [706]:
import numpy as np
import pandas as pd
from pathlib import Path
filepath_source = Path('data/raw/disaster/emdat_public_2022_12_22_full.xlsx')
filepath_all = Path("data/processed/disaster/disaster-all.csv")
filepath_global = Path("data/processed/disaster/disaster-global.csv")
filepath_country = Path('data/processed/disaster/disaster-country.csv')
filepath_region = Path("data/processed/disaster/disaster-region.csv")

In [707]:
dis = pd.read_excel(filepath_source, skiprows = 6)

/Users/paulschmitt/miniforge3/envs/dopp-a2/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## First Look

In [708]:
dis.head()

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
2,1901-0003-BEL,1901,3,NaN,Technological,Technological,Industrial accident,Explosion,NaN,Coal mine,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
3,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,781207.0,3.200175,NaN,NaN,NaN,NaN
4,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN


## Reformat Attribute-Names

1. Replace whitespaces with underscores
2. Convert every character to lowercase
3. Rename specific columns to ensure uniformity

In [709]:
# Remove whitespaces from all col-names and convert them to lower-case
dis.columns = [c.replace(' ', '_').lower() for c in dis.columns]
dis.rename(columns={'country':'country_name', 'iso':'country_code', 'disaster_subtype':'subtype', 'disaster_type':'type', 'total_deaths':'deaths'}, inplace=True)

## Filter for all relevant attributes & observations

1. We only consider observations of disasters of type natural. (rows)
2. We only consider relevant attributes. (columns)

## Which disaster groups are present in the dataset ?

In [710]:
dis.disaster_group.unique()

array(['Natural', 'Technological', 'Complex Disasters'], dtype=object)

We only focus on disasters which have a natural causation

In [711]:
dis = dis[dis.disaster_group == "Natural"]

Which types of natural disasters are there ?

In [712]:
dis["type"].unique()

array(['Drought', 'Earthquake', 'Volcanic activity',
       'Mass movement (dry)', 'Storm', 'Flood', 'Epidemic', 'Landslide',
       'Wildfire', 'Extreme temperature ', 'Fog', 'Insect infestation',
       'Impact', 'Animal accident', 'Glacial lake outburst'], dtype=object)

The types of disasters are mostly the ones a normal person would expect when thinking about natural disasters.
But there are some strange types like insect-infestations or animal-accident which are not that obvious to understand.
Therefore, we need to take a closer look at the disasters of those types.

In [713]:
dis[dis.type == "Insect infestation"].deaths.sum()

0.0

It is safe to say that we can omit observations of disasters of type insect-infestations since there are no accounts of people dying from those kinds of incidents.

In [714]:
dis[dis.type == "Impact"].deaths.sum()

0.0

The same goes for disaster type impact, which only occurred once in Russia with again 0 deaths.
We omit this event.

In [715]:
dis[dis.type == "Animal accident"].deaths.count()

1

We also decided to omit disasters of type animal-accident since there is only one recorded accident over the last 100 years with only 12 people dying. Therefore, it makes not much sense to include it into our further research.

In [716]:
dis = dis[((dis.type != "Insect infestation") & (dis.type != "Animal accident")) & (dis.type != "Impact")]

We also decided to no include epidemics since it would go beyond the scope of this task.

In [717]:
dis = dis[dis.type != "Epidemic"]

We only consider the following types of disasters

In [718]:
dis["type"].unique()

array(['Drought', 'Earthquake', 'Volcanic activity',
       'Mass movement (dry)', 'Storm', 'Flood', 'Landslide', 'Wildfire',
       'Extreme temperature ', 'Fog', 'Glacial lake outburst'],
      dtype=object)

In [719]:
dis[dis.type == "Impact"].deaths.head()

Series([], Name: deaths, dtype: float64)

Now we take a look at the different attributes of each observation

In [720]:
dis.dtypes

dis_no                                        object
year                                           int64
seq                                            int64
glide                                         object
disaster_group                                object
disaster_subgroup                             object
type                                          object
subtype                                       object
disaster_subsubtype                           object
event_name                                    object
country_name                                  object
country_code                                  object
region                                        object
continent                                     object
location                                      object
origin                                        object
associated_dis                                object
associated_dis2                               object
ofda_response                                 

In [721]:
# Disaster-All
dis_all_col_names = ["year", "dis_no", "region", "continent", "country_name", "country_code", "location",                             "type", "subtype", "deaths", "dis_mag_value", "dis_mag_scale", "start_year", "end_year"]
dis_all = dis.filter(items=dis_all_col_names)

## Are there missing values?

Fill missing Values for the number of deaths

We can assume that missing values for the number of deaths of a particular disaster means that the deathtoll was 0.

For the subtype we take a look for which type of natural disasters a subtype is not provided.

In [722]:
for col in dis_all:
    print(col + ": " + str(dis_all.loc[:, col].isnull().sum()))
print("Total: " + str(len(dis_all)))

year: 0
dis_no: 0
region: 0
continent: 0
country_name: 0
country_code: 0
location: 1449
type: 0
subtype: 3116
deaths: 4412
dis_mag_value: 9903
dis_mag_scale: 1114
start_year: 0
end_year: 0
Total: 14871


In [723]:
dis_all[dis_all["subtype"].isna()]["type"].unique()

array(['Flood', 'Storm', 'Landslide', 'Wildfire', 'Fog',
       'Mass movement (dry)', 'Volcanic activity', 'Drought',
       'Earthquake', 'Glacial lake outburst'], dtype=object)

Unfortunately the missing values in the subtype column do not correspond to specific types of disasters.
We can not conclude that easily what caused the values to be missing.

In [724]:
dis_all[['deaths']] = dis_all[['deaths']].fillna(value=0)

## ISO-Codes

Compare iso-codes to match the id's of each row with the other datasets.
We use the ISO3-Codes, which contain 3 letters to identify a country.
From now on, if ISO-Codes are mentioned, we are always speaking of ISO3-Codes.

In [725]:
un_country_codes = pd.read_csv("data/raw/country-codes/un-country-codes.csv", sep=";")
un_country_codes.columns = [c.replace(' ', '_').replace('-','_') for c in un_country_codes.columns]

In [726]:
countries_with_iso = dis_all.merge(un_country_codes, how="left", left_on='country_name', right_on='Country_or_Area')[["country_name", "country_code", "ISO_alpha3_Code"]]

In [727]:
countries_with_iso.head(10)

,country_name,country_code,ISO_alpha3_Code
0,Cabo Verde,CPV,CPV
1,India,IND,IND
2,Guatemala,GTM,GTM
3,Guatemala,GTM,GTM
4,Guatemala,GTM,GTM
5,Canada,CAN,CAN
6,Comoros (the),COM,NaN
7,Bangladesh,BGD,BGD
8,Canada,CAN,CAN
9,India,IND,IND


## Display all countries for which NO matching ISO-Code was found

In [728]:
mismatches = countries_with_iso[countries_with_iso.ISO_alpha3_Code.isnull()]
mismatches.country_name.unique()

array(['Comoros (the)', 'Hong Kong', 'Gambia (the)', 'Germany Fed Rep',
       'Bahamas (the)', 'Dominican Republic (the)', 'Cook Islands (the)',
       'Azores Islands',
       'United Kingdom of Great Britain and Northern Ireland (the)',
       'Netherlands Antilles', 'Congo (the)', 'Czechoslovakia',
       'United States of America (the)', 'Soviet Union', 'Niger (the)',
       'Turkey', 'Philippines (the)', 'Taiwan (Province of China)',
       'Korea (the Republic of)', 'Netherlands (the)', 'Canary Is',
       'Tanzania, United Republic of',
       "Lao People's Democratic Republic (the)", 'Sudan (the)',
       'Yemen Arab Rep', 'Yugoslavia', 'Wallis and Futuna',
       'Congo (the Democratic Republic of the)', 'Yemen P Dem Rep',
       'Germany Dem Rep', "Korea (the Democratic People's Republic of)",
       'Turks and Caicos Islands (the)', 'Marshall Islands (the)',
       'Russian Federation (the)',
       'Macedonia (the former Yugoslav Republic of)', 'Macao',
       'Virgin Isla

## Rename remaining Countrynames to a standardized format

In [729]:
# Replace suffix
dis_all['country_name'] = dis_all['country_name'].apply(lambda x: x.replace(' (the)', ''))
# Reorder compley country-names
dis_all['country_name'] = dis_all['country_name'].apply(lambda x: x.split(',')[1] + " " + x.split(',')[0] if ',' in x else x)
# Remove random white-spaces at the start of a name
dis_all['country_name'] = dis_all['country_name'].apply(lambda x: x[1:] if x.startswith(' ') else x)


In [730]:
new_country_names = {
    "Germany Fed Rep": "Germany",
    "Germany Dem Rep": "Germany",
    "Hong Kong": "China",
    "Macao": "China",
    "Åland": "Åland Islands",
    "Congo (the Democratic Republic of the)": "Congo",
    "Turkey": "Türkiye",
    "Korea (the Republic of)": "Republic of Korea",
    "Macedonia (the former Yugoslav Republic of)": "North Macedonia",
    "Congo (Democratic Republic of the)": "Democratic Republic of the Congo",
    "Yemen P Dem Rep": "Yemen",
    "Yemen Arab Rep": "Yemen",
    "Korea (the Democratic People's Republic of)": "Democratic People's Republic of Korea",
    "Korea (the Republic of)": "Republic of Korea",
    "Serbia Montenegro" : "Serbia",
    "Moldova (the Republic of)" : "Republic of Moldova",
    "Czech Republic" : "Czechia",
    "Taiwan (Province of China)" : "Taiwan"   
}

In [731]:
dis_all = dis_all.replace({"country_name": new_country_names}, inplace=False)

## Check for countries with missing ISO-Codes

In [732]:
countries_with_iso = dis_all.merge(un_country_codes, how="left", left_on='country_code', right_on='ISO_alpha3_Code')[["country_name", "country_code", "ISO_alpha3_Code"]]
mismatches = countries_with_iso[countries_with_iso.ISO_alpha3_Code.isnull()]
mismatches.country_name.unique()

array(['Germany', 'Azores Islands', 'Netherlands Antilles',
       'Czechoslovakia', 'Soviet Union', 'Taiwan', 'Canary Is', 'Yemen',
       'Yugoslavia', 'Serbia'], dtype=object)

In [733]:
countries_with_iso.head(10)
mask = countries_with_iso.notnull().all(axis=1)
countries_with_iso[~mask].country_name.unique()

array(['Germany', 'Azores Islands', 'Netherlands Antilles',
       'Czechoslovakia', 'Soviet Union', 'Taiwan', 'Canary Is', 'Yemen',
       'Yugoslavia', 'Serbia'], dtype=object)

## Assign ISO-Codes (we know of) to countries

In [734]:
dis_all.loc[dis_all.country_name == "Germany", "country_code"] = "DEU"
dis_all.loc[dis_all.country_name == "Serbia", "country_code"] = "SRB"
dis_all.loc[dis_all.country_name == "Yemen", "country_code"] = "YEM"
dis_all.loc[dis_all.country_name == "Taiwan", "country_code"] = "TWM"
dis_all.loc[dis_all.country_name == "Canary Is", "country_code"] = "SPI"
dis_all.loc[dis_all.country_name == "Azores Islands", "country_code"] = "AZO"

## Check which countries still do not have an ISO-Code

The following countries either do not have an ISO-Code assigned because they do not exist anymore or are not recognized internationally.

For small countries like Azore-Islands or the netherlands antilles it is not that tragic, since they probably contribute only marginally to the total number of deaths by natural disasters globally or for a specific region.
They are therefore negligible.

For internationally unrecognized countries (Taiwan) we can default to a specifically assigned ISO-Code by us.

The difficult part is to make sense of the observations belonging to a larger country which has been split up into smaller nations in the last 100 years. (Soviet Union, Czechoslovakia, Yugoslavia)

In [735]:
countries_with_iso = dis_all.merge(un_country_codes, how="left", left_on='country_code', right_on='ISO_alpha3_Code')[["country_name", "country_code", "ISO_alpha3_Code"]]
mismatches = countries_with_iso[countries_with_iso.ISO_alpha3_Code.isnull()]
mismatches.country_name.unique()

array(['Azores Islands', 'Netherlands Antilles', 'Czechoslovakia',
       'Soviet Union', 'Taiwan', 'Canary Is', 'Yugoslavia'], dtype=object)

## Taiwan: Assign ISO-Code manually (TWN)

In [736]:
dis_all.loc[dis_all.country_name == "Taiwan", "country_code"] = "TWN"

## Count all disasters that happened in the Soviet Union

In [737]:
dis_all[dis_all.country_code == "SUN"].deaths.count()

62

## Count all disasters that happened in Czechoslovakia

In [738]:
dis_all[dis_all.country_code == "CSK"].deaths.count()

9

## Count all disasters that happened in Yugoslavia

In [739]:
dis_all[dis_all.country_code == "YUG"].deaths.count()

21

## Check in which part of the country the disaster occured

In [740]:
dis_all[dis_all.country_code == "CSK"].location

970     Tatra montains (Slovakia)
2797                          NaN
2798                          NaN
2854                          NaN
3012                          NaN
3115    Tatra Mountain (Slovakia)
3116                          NaN
3254                     Slovakia
3342                          NaN
Name: location, dtype: object

## Determine Location

To determine in which currently existing country those disasters happened,
we need to take a look at the location-attribute

Fortunately only the disasters in Czechoslovakia have missing values for the location attribute.
For all disasters in the other dissolved countries an exact location is provided.

Now we proceed by checking matching the location with the now existing countries that were part of the former nations.

Soviet Union (SUN):
- Armenia
- Azerbaijan
- Belarus
- Estonia
- Georgia
- Kazakhstan
- Kyrgyzstan
- Latvia
- Lithuania
- Moldova
- Russia
- Tajikistan
- Turkmenistan
- Ukraine
- Uzbekistan

Yugoslavia (YUG):
- Bosnia and Herzegovina
- Croatia
- Kosovo (included but not part of the dataset)
- Montenegro
- North Macedonia
- Serbia

Czechoslovakia (CSK):
- The Czech Republic
- Slovakia

In [741]:
former_sum_country_names = ["Russian Federation", "Armenia", "Azerbaijan", "Belarus", "Estonia", "Georgia", "Kazakhstan", "Kyrgyzstan", "Latvia", "Lithuania", "Moldova", "Tajikistan", "Turkmenistan", "Ukraine", "Uzbekistan"]

former_yug_country_names = ["Bosnia and Herzegovina", "Croatia", "Kosovo", "Montenegro", "North Macedonia","Serbia"]

former_csk_country_names = ["Czechia", "Slovakia"]

In [742]:
dis_all[dis_all.country_code == "SUN"].location

1198                        Andishan,Tashkent (Uzbekistan)
1262     South Ukraine, Volga, Ural (Kazakhstan,Russian...
1274                           Rostov (Russian Federation)
1430                                          Turkmenistan
1447                               Ashkabat (Turkmenistan)
                               ...                        
7169       Ulan-Ude region (Buryatia) (Russian Federation)
7176                Volgograd Region (Russian Federation) 
24810                                 Semacha (Azerbaijan)
24819           Usoy, Barchidiv, Posor, Nisur (Tajikistan)
24829                                Dushanbe (Tajikistan)
Name: location, Length: 62, dtype: object

In [743]:
dis_sun_with_region = dis_all[dis_all.country_code == "SUN"].copy()
dis_sun_with_region["region"] = np.nan

mask_europe = dis_sun_with_region["location"].str.contains("Russian Federation|Ukraine|Moldavia|Siberia").fillna(False)
mask_asia = dis_sun_with_region["location"].str.contains("Kazakhstan|Azerbaijan|Uzbekistan|Turkmenistan|Georgia|Armenia|Kyrgystan|Tajikistan|Tajiskistan|Tadzhikistan|Tadjikistan|Caucasus region|Dushanbe", case=False).fillna(False)

# disasters in europe and asia
dis_sun_with_region[mask_europe & mask_asia]

,year,dis_no,region,continent,country_name,country_code,location,type,subtype,deaths,dis_mag_value,dis_mag_scale,start_year,end_year
1262,1921,1921-9001-SUN,NaN,Europe,Soviet Union,SUN,"South Ukraine, Volga, Ural (Kazakhstan,Russian...",Drought,Drought,1200000.0,NaN,Km2,1921,1921


Only one Event happened in both the asian as well as the european part of the soviet union.
It is also a major event since it is a drought which caused the death of 1.2 million people.

Researching the details of this event one can conclude that this observation can only be the Russian famine of 1921–1922.
It mostly affected people living in europe, hence we assign this single observation the region europe.
(https://en.wikipedia.org/wiki/Russian_famine_of_1921%E2%80%931922)

For all other observations, the region should be unambiguous.

In [744]:
dis_sun_with_region.loc[mask_europe, "region"] = "Europe"
dis_sun_with_region.loc[mask_asia, "region"] = "Asia"
dis_sun_with_region.loc[1262, "region"] = "Europe"

The observations which still have missing region values, all have no recorded deaths and can therefore be safely ignored.

In [745]:
# Czechoslovakia
dis_all[dis_all.country_code == "CSK"].region

970     Eastern Europe
2797    Eastern Europe
2798    Eastern Europe
2854    Eastern Europe
3012    Eastern Europe
3115    Eastern Europe
3116    Eastern Europe
3254    Eastern Europe
3342    Eastern Europe
Name: region, dtype: object

In [746]:
# Yugoslavia
dis_all[dis_all.country_code == "YUG"].region

2509     Southern Europe
2518     Southern Europe
2521     Southern Europe
2539     Southern Europe
2544     Southern Europe
2555     Southern Europe
4720     Southern Europe
4746     Southern Europe
4748     Southern Europe
4751     Southern Europe
4810     Southern Europe
4811     Southern Europe
4889     Southern Europe
4890     Southern Europe
4987     Southern Europe
4988     Southern Europe
5035     Southern Europe
5189     Southern Europe
7134     Southern Europe
7136     Southern Europe
24847    Southern Europe
Name: region, dtype: object

## Save the Disaster-All file

In [747]:
dis_all.to_csv(filepath_all)

## Create/Save the Disaster-Country file

In [748]:
dis_country_col_names = ["year", "country_name", "country_code", "type", "subtype", "deaths"]
dis_country = dis_all.filter(items=dis_country_col_names)
dis_country.to_csv(filepath_country)

In [749]:
dis_country.head(10)

,year,country_name,country_code,type,subtype,deaths
0,1900,Cabo Verde,CPV,Drought,Drought,11000.0
1,1900,India,IND,Drought,Drought,1250000.0
3,1902,Guatemala,GTM,Earthquake,Ground movement,2000.0
4,1902,Guatemala,GTM,Volcanic activity,Ash fall,1000.0
5,1902,Guatemala,GTM,Volcanic activity,Ash fall,6000.0
6,1903,Canada,CAN,Mass movement (dry),Rockfall,76.0
7,1903,Comoros,COM,Volcanic activity,Ash fall,17.0
10,1904,Bangladesh,BGD,Storm,Tropical cyclone,0.0
12,1905,Canada,CAN,Mass movement (dry),Rockfall,18.0
13,1905,India,IND,Earthquake,Ground movement,20000.0


## Create/Save the Disaster-Region file

In [750]:
dis_region_col_names = ["year", "region_name", "region_code", "iso", "type", "subtype", "deaths"]
dis_region = dis_all.filter(items=dis_region_col_names)
dis_region.to_csv(filepath_region)

In [751]:
dis_region.head(10)

,year,type,subtype,deaths
0,1900,Drought,Drought,11000.0
1,1900,Drought,Drought,1250000.0
3,1902,Earthquake,Ground movement,2000.0
4,1902,Volcanic activity,Ash fall,1000.0
5,1902,Volcanic activity,Ash fall,6000.0
6,1903,Mass movement (dry),Rockfall,76.0
7,1903,Volcanic activity,Ash fall,17.0
10,1904,Storm,Tropical cyclone,0.0
12,1905,Mass movement (dry),Rockfall,18.0
13,1905,Earthquake,Ground movement,20000.0


## Create/Save the Disaster-global file

In [752]:
dis_global_attributes = ["year", "type", "subtype", "deaths"]
dis_global = dis_all[dis_global_attributes]
dis_global.to_csv(filepath_global)

In [753]:
dis_region.head(10)

,year,type,subtype,deaths
0,1900,Drought,Drought,11000.0
1,1900,Drought,Drought,1250000.0
3,1902,Earthquake,Ground movement,2000.0
4,1902,Volcanic activity,Ash fall,1000.0
5,1902,Volcanic activity,Ash fall,6000.0
6,1903,Mass movement (dry),Rockfall,76.0
7,1903,Volcanic activity,Ash fall,17.0
10,1904,Storm,Tropical cyclone,0.0
12,1905,Mass movement (dry),Rockfall,18.0
13,1905,Earthquake,Ground movement,20000.0
